In [ ]:
from IPython.display import display
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from onekey_algo import OnekeyDS as okds
import pandas as pd

os.makedirs('img', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('features', exist_ok=True)
# 设置数据目录
#label_file = r'你自己数据的路径'
feature_file = r'C:\Users\Admin\Desktop\T and V\HCC.csv'
# 对应的标签文件
label_file = r'C:\Users\Admin\Desktop\T and V\label.csv'
#label_file = os.path.join(okds.ct, 'label_multicenter.csv')
# 读取标签数据列名
labels = ['label']

feature_data = pd.read_csv(feature_file)
display(feature_data)
label_data = pd.read_csv(label_file)
label_data.head()

from onekey_algo.custom.utils import print_join_info
# 删掉ID这一列。
print_join_info(feature_data, label_data)
combined_data = pd.merge(feature_data, label_data, on=['ID'], how='inner')
ids = combined_data['ID']
combined_data = combined_data.drop(['ID'], axis=1)
print(combined_data[labels].value_counts())
combined_data.columns

from onekey_algo.custom.components.comp1 import normalize_df
data = normalize_df(combined_data, not_norm=labels, group='group')
data = data.dropna(axis=1)
data.describe()

pearson_corr = data[data['group'] == 'train'][[c for c in data.columns if c not in labels]].corr('pearson')

import seaborn as sns
import matplotlib.pyplot as plt
from onekey_algo.custom.components.comp1 import draw_matrix

if combined_data.shape[1] < 100:
    plt.figure(figsize=(10.0, 8.0))
    # 选择可视化的相关系数
    draw_matrix(pearson_corr, annot=True, cmap='YlGnBu', cbar=False)
    plt.savefig(f'img/feature_corr.svg', bbox_inches = 'tight')
    
import seaborn as sns
import matplotlib.pyplot as plt

if combined_data.shape[1] < 100:
    pp = sns.clustermap(pearson_corr, linewidths=.5, figsize=(50.0, 40.0), cmap='YlGnBu')
    plt.setp(pp.ax_heatmap.get_yticklabels(), rotation=0)
    plt.savefig(f'img/feature_cluster.svg', bbox_inches = 'tight')
    
from onekey_algo.custom.components.comp1 import select_feature
sel_feature = select_feature(pearson_corr, threshold=0.9, topn=32, verbose=False)

sel_feature = sel_feature + labels + ['group']
sel_feature

sel_data = data[sel_feature]
sel_data.describe()

alpha = okcomp.comp1.lasso_cv_coefs(X_data, y_data, column_names=None, alpha_logmin=-3)
plt.savefig(f'img/feature_lasso.svg', bbox_inches = 'tight')

okcomp.comp1.lasso_cv_efficiency(X_data, y_data, points=50, alpha_logmin=-3)
plt.savefig(f'img/feature_mse_label.svg', bbox_inches = 'tight')

from sklearn import linear_model

models = []
for label in labels:
    clf = linear_model.Lasso(alpha=alpha)
    clf.fit(X_data, y_data[label])
    models.append(clf)
    
COEF_THRESHOLD = 1e-8 # 筛选的特征阈值
scores = []
selected_features = []
for label, model in zip(labels, models):
    feat_coef = [(feat_name, coef) for feat_name, coef in zip(column_names, model.coef_) 
                 if COEF_THRESHOLD is None or abs(coef) > COEF_THRESHOLD]
    selected_features.append([feat for feat, _ in feat_coef])
    formula = ' '.join([f"{coef:+.6f} * {feat_name}" for feat_name, coef in feat_coef])
    score = f"{label} = {model.intercept_} {'+' if formula[0] != '-' else ''} {formula}"
    scores.append(score)
    
print(scores[0])


feat_coef = sorted(feat_coef, key=lambda x: x[1])
feat_coef_df = pd.DataFrame(feat_coef, columns=['feature_name', 'Coefficients'])
feat_coef_df.plot(x='feature_name', y='Coefficients', kind='barh')

plt.savefig(f'img/feature_weights.svg', bbox_inches = 'tight')

feat_coef_df.to_csv('TT5_features.csv',index=False)
X_data.to_csv('SHAP_features.csv',index=False)

model_names = ['SVM', 'KNN', 'RandomForest',  'XGBoost', 'LightGBM', 'NaiveBayes', 'AdaBoost', 'GradientBoosting', 'LR', 'MLP']
models = okcomp.comp1.create_clf_model(model_names)
model_names = list(models.keys())


import joblib

from onekey_algo.custom.components.comp1 import plot_feature_importance, smote_resample
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from onekey_algo.custom.components.comp1 import plot_feature_importance, plot_learning_curve, smote_resample

targets = []
os.makedirs('models', exist_ok=True)
for l in labels:
    new_models = okcomp.comp1.create_clf_model_none_overfit(model_names)
    new_models['LR'] = LogisticRegression(penalty='none', max_iter=100)
    new_models['SVM'] = SVC(penalty=100,gamma='auto',kernel='linear')
    new_models['RandomForest'] = RandomForestClassifier(n_estimators=50, max_depth=4) 
    new_models['KNN'] = KNeighborsClassifier(n_neighbors=5, weights='uniform',metric='euclidean')
    new_models['XGBoost'] = XGBClassifier(n_estimators=6, objective='binary:logistic', max_depth=3, min_child_weight=0.2,use_label_encoder=False, eval_metric='error')
    new_models['AdaBoost'] = AdaBoostClassifier(n_estimators=50, learning_rate=0.1,base_estimator__max_depth=3)
    new_models['LightGBM'] = LGBMClassifier(n_estimators=20, max_depth=4, min_child_weight=0.5,)
    new_models['NaiveBayes'] =GaussianNB(alpha=1)
    new_models['GradientBoosting'] = GradientBoostingClassifier(n_estimators=200,max_depth=4, learning_rate=0.1, min_samples_split=4,min_samples_leaf=1, max_features='sqrt')
    new_models['MLP'] = MLPClassifierr(hidden_layer_sizes=(50, 50),activation='relu',solver='sgd',alpha= 0.01,learning_rate_init=0.01)
#模型筛选
    
    model_names = list(new_models.keys())
    new_models = list(new_models.values())
    for mn, m in zip(model_names, new_models):
        X_train_smote, y_train_smote = X_train_sel, y_train_sel
        # 取消下一行的注释可以使用Smote进行采样，解决样本不均衡的问题。
#         X_train_smote, y_train_smote = smote_resample(X_train_sel, y_train_sel)
        m.fit(X_train_smote, y_train_smote[l])
        # 保存训练的模型
        joblib.dump(m, f'models/{mn}_{l}.pkl') 
        # 输出模型特征重要性，只针对高级树模型有用
        plot_feature_importance(m, selected_features[0], save_dir='img')
        
#         plot_learning_curve(m, X_train_sel, y_train_sel, title=f'Learning Curve {mn}')
#         plt.savefig(f"img/Rad_{mn}_learning_curve.svg", bbox_inches='tight')
        plt.show()
    targets.append(new_models)
    
    
    
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from onekey_algo.custom.components.delong import calc_95_CI
from onekey_algo.custom.components.metrics import analysis_pred_binary

predictions = [[(model.predict(X_train_sel), model.predict(X_test_sel)) 
                for model in target] for label, target in zip(labels, targets)]
pred_scores = [[(model.predict_proba(X_train_sel), model.predict_proba(X_test_sel)) 
                for model in target] for label, target in zip(labels, targets)]

metric = []
pred_sel_idx = []
for label, prediction, scores in zip(labels, predictions, pred_scores):
    pred_sel_idx_label = []
    for mname, (train_pred, test_pred), (train_score, test_score) in zip(model_names, prediction, scores):
        # 计算训练集指数
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_train_sel[label], 
                                                                                              train_score[:, 1])
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"{label}-train"))
                 
        # 计算验证集指标
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_test_sel[label], 
                                                                                              test_score[:, 1])
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"{label}-test"))
        # 计算thres对应的sel idx
        pred_sel_idx_label.append(np.logical_or(test_score[:, 0] >= thres, test_score[:, 1] >= thres))
    
    pred_sel_idx.append(pred_sel_idx_label)
metric = pd.DataFrame(metric, index=None, columns=['model_name', 'Accuracy', 'AUC', '95% CI',
                                                   'Sensitivity', 'Specificity', 
                                                   'PPV', 'NPV', 'Precision', 'Recall', 'F1',
                                                   'Threshold', 'Task'])

sel_model = model_names

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
    
        # Plot all ROC curves
        plt.figure(figsize=(8, 8))
        for pred_score, label in zip(pred_scores, labels):
            okcomp.comp1.draw_roc([np.array(y_train_sel[label]), np.array(y_test_sel[label])], 
                                  pred_score[sel_model_idx], 
                                  labels=['Training', 'Internal validation'], title=f"Model: {sm}")
            plt.savefig(f'img/model_{sm}_roc.svg', bbox_inches = 'tight')
            
sel_model = model_names

for pred_score, label in zip(pred_scores, labels):
    pred_test_scores = []
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            pred_test_scores.append(pred_score[sel_model_idx][1])
    okcomp.comp1.draw_roc([np.array(y_test_sel[label])] * len(pred_test_scores), 
                          pred_test_scores, 
                          labels=sel_model, title=f"Internal validation cohort AUC")
    plt.savefig(f'img/model_roc.svg', bbox_inches = 'tight')
    
sel_model = model_names


from onekey_algo.custom.components.comp1 import plot_DCA

for pred_score, label in zip(pred_scores, labels):
    pred_test_scores = []
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            okcomp.comp1.plot_DCA(pred_score[sel_model_idx][1][:,1], np.array(y_test_sel[label]),
                                  title=f'{sm} DCA')
            plt.savefig(f'img/model_{sm}_dca.svg', bbox_inches = 'tight')
    
# 设置绘制参数
sel_model = model_names
c_matrix = {}

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            cm = okcomp.comp1.calc_confusion_matrix(predictions[idx][sel_model_idx][-1], y_test_sel[label],
#                                                     sel_idx = pred_sel_idx[idx][sel_model_idx],
                                                    class_mapping={1:'MVI+', 0:'MVI-'}, num_classes=2)
            c_matrix[label] = cm
            plt.figure(figsize=(5, 4))
            plt.title(f'Rad Model:{sm}')
            okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.3g')
            plt.savefig(f'img/model_{sm}_cm.svg', bbox_inches = 'tight')
            
sel_model = model_names
c_matrix = {}

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):            
            okcomp.comp1.draw_predict_score(pred_scores[idx][sel_model_idx][-1], y_test_sel[label])
            plt.title(f'{sm} sample predict score')
            plt.legend(labels=["label=MVI-","label=MVI+"],loc="lower right") 
            plt.savefig(f'img/model_{sm}_sample_dis.svg', bbox_inches = 'tight')
            plt.show()
            
            
import os
import numpy as np

os.makedirs('results', exist_ok=True)
sel_model = sel_model

for idx, label in enumerate(labels):
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            target = targets[idx][sel_model_idx]
            # 预测训练集和测试集数据。
            train_indexes = np.reshape(np.array(train_ids), (-1, 1)).astype(str)
            test_indexes = np.reshape(np.array(test_ids), (-1, 1)).astype(str)
            y_train_pred_scores = target.predict_proba(X_train_sel)
            y_test_pred_scores = target.predict_proba(X_test_sel)
            columns = ['ID'] + [f"{label}-{i}"for i in range(y_test_pred_scores.shape[1])]
            # 保存预测的训练集和测试集结果
            result_train = pd.DataFrame(np.concatenate([train_indexes, y_train_pred_scores], axis=1), columns=columns)
            result_train.to_csv(f'results/{sm}_Rad_training.csv', index=False)
            result_test = pd.DataFrame(np.concatenate([test_indexes, y_test_pred_scores], axis=1), columns=columns)
            result_test.to_csv(f'results/{sm}_Rad_test.csv', index=False)
            
            
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

# 读取数据
data = pd.read_csv('C:\\Users\\Admin\\Desktop\\train\\HCC_all_corhort.csv')

# 分离特征和目标变量
X = data.iloc[:, 1:]  # 从第二列开始是特征
y = data.iloc[:, 0]   # 第一列是目标变量

# SMOTE重采样
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 初始化10折交叉验证
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# 存储每一折的结果
auc_scores = []

# 交叉验证
for fold, (train_index, test_index) in enumerate(skf.split(X_resampled, y_resampled), start=1):
    X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled.iloc[test_index]
    
    # 初始化GradientBoosting模型
    model = GradientBoostingClassifier(random_state=42)
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 预测概率
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 计算AUC
    auc = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append({'Fold': fold, 'AUC': auc})

# 将结果转换为DataFrame
results_df = pd.DataFrame(auc_scores)

# 保存到CSV文件
output_path_csv = 'C:\\Users\\Admin\\Desktop\\AUC_Scores.csv'
results_df.to_csv(output_path_csv, index=False)

# 提取AUC分数列表
auc_scores_list = [item['AUC'] for item in auc_scores]

# 计算平均AUC和标准差
mean_auc = np.mean(auc_scores_list)
std_auc = np.std(auc_scores_list)

# 绘制AUC曲线
plt.figure(figsize=(10, 6))
plt.plot(auc_scores_list, marker='o', linestyle='-', color='b')
plt.fill_between(range(1, len(auc_scores_list)+1), [mean_auc - std_auc]*len(auc_scores_list), [mean_auc + std_auc]*len(auc_scores_list), color='b', alpha=0.2)
plt.title('10-Fold Cross-Validation AUC Scores of Training Cohort')
plt.xlabel('Fold')
plt.ylabel('AUC Score')
plt.grid(True)
plt.savefig('C:\\Users\\Admin\\Desktop\\AUC_Curve.svg')
plt.show()


import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.preprocessing import label_binarize
from statsmodels.stats.proportion import proportion_confint
import numpy as np

# 加载数据
data = pd.read_csv('C:/Users/Admin/Desktop/K折交叉验证结果/DeLong_Test/Training.csv')

# 目标分类变量和两个预测变量
y = data['label']
y_pred1 = data.iloc[:, 1]  # 第二列变量
y_pred2 = data.iloc[:, 2]  # 第三列变量

# 计算AUC
auc1 = roc_auc_score(y, y_pred1)
auc2 = roc_auc_score(y, y_pred2)

# 计算敏感性、特异性、阳性预测值、阴性预测值及其95%置信区间
fpr1, tpr1, _ = roc_curve(y, y_pred1)
fpr2, tpr2, _ = roc_curve(y, y_pred2)

# 计算最佳阈值（Youden's J statistic）
j_statistic1 = tpr1 - fpr1
j_index1 = np.argmax(j_statistic1)
threshold1 = fpr1[j_index1]

j_statistic2 = tpr2 - fpr2
j_index2 = np.argmax(j_statistic2)
threshold2 = fpr2[j_index2]

# 计算敏感性、特异性、阳性预测值、阴性预测值
sensitivity1 = tpr1[j_index1]
specificity1 = 1 - fpr1[j_index1]
ppv1 = sensitivity1 / (sensitivity1 + (1 - specificity1) * (1 - threshold1))
npv1 = specificity1 / (specificity1 + threshold1 * (1 - sensitivity1))

sensitivity2 = tpr2[j_index2]
specificity2 = 1 - fpr2[j_index2]
ppv2 = sensitivity2 / (sensitivity2 + (1 - specificity2) * (1 - threshold2))
npv2 = specificity2 / (specificity2 + threshold2 * (1 - sensitivity2))

# 计算95%置信区间
conf_int1 = proportion_confint(ppv1, ppv1 + npv1, method='wilson')
conf_int2 = proportion_confint(ppv2, ppv2 + npv2, method='wilson')

# 打印结果
print(f'AUC1: {auc1:.3f}, 95% CI: {conf_int1}')
print(f'AUC2: {auc2:.3f}, 95% CI: {conf_int2}')
print(f'Sensitivity1: {sensitivity1:.3f}, Specificity1: {specificity1:.3f}, PPV1: {ppv1:.3f}, NPV1: {npv1:.3f}')
print(f'Sensitivity2: {sensitivity2:.3f}, Specificity2: {specificity2:.3f}, PPV2: {ppv2:.3f}, NPV2: {npv2:.3f}')

# DeLong检验比较两个AUC
from scipy.stats import norm

def delong_test(auc1, auc2, n1, n2):
    se1 = np.sqrt((auc1 * (1 - auc1) / n1) + (auc2 * (1 - auc2) / n2))
    se2 = np.sqrt((auc2 * (1 - auc2) / n2) + (auc1 * (1 - auc1) / n1))
    z = (auc1 - auc2) / np.sqrt(se1**2 + se2**2)
    p = 2 * (1 - norm.cdf(abs(z)))
    return z, p

z, p = delong_test(auc1, auc2, len(y), len(y))
print(f'DeLong Test: Z = {z:.3f}, P = {p:.3f}')


import matplotlib.pyplot as plt
import os

# 绘制ROC曲线
plt.figure(figsize=(8, 8))
plt.plot(fpr1, tpr1, label=f'Combined model AUC = {auc1:.3f}')
plt.plot(fpr2, tpr2, label=f'Clinical model AUC = {auc2:.3f}')
plt.plot([0, 1], [0, 1], 'k--')  # 绘制对角线
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Test cohorts curves')
plt.legend(loc='lower right')

# 保存ROC曲线到桌面
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
roc_plot_path = os.path.join(desktop_path, 'ROC_Curves.svg')
plt.savefig(roc_plot_path)
plt.close()

print(f'ROC curves saved to: {roc_plot_path}')

# SHAP可视化
import numpy as np
import shap
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os


model_path = r'D:\onekey\onekey_comp\comp2-结构化数据\models/GradientBoosting_label.pkl'
feature_path = r'D:/onekey\onekey_comp/comp2-结构化数据/SHAP_features.csv'
analysis_label = 1
analysis_sample = 150
shap_value = get_shap_value(shap_values, analysis_label=analysis_label, samples=analysis_sample)
shap.plots.waterfall(shap_value, max_display=10, show=False)
plt.savefig(f"img/{analysis_sample}_waterfall.svg", bbox_inches='tight')
plt.show()
shap_value.feature_names = [f.replace('_', '\n') + '\n' for f in shap_value.feature_names]
shap.plots.force(shap_value, show=False, matplotlib=True, figsize=(20, 4))
plt.savefig(f"img/{analysis_sample}_force.svg", bbox_inches='tight')
plt.show()
shap_value = get_shap_value(shap_values, analysis_label=analysis_label, samples=analysis_sample)
shap.plots.waterfall(shap_value, max_display=10, show=False)
plt.savefig(f"img/{analysis_sample}_waterfall.svg", bbox_inches='tight')
plt.show()
shap_value.feature_names = [f.replace('_', '\n') + '\n' for f in shap_value.feature_names]
shap.plots.force(shap_value, show=False, matplotlib=True, figsize=(20, 4))
plt.savefig(f"img/{analysis_sample}_force.svg", bbox_inches='tight')
plt.show() 